In [ ]:
from IPython.display import HTML
from google.colab import output
import requests

# ngrok로 얻은 주소 입력
LMSTUDIO_API_URL = "https://b2f1ea18ceb9.ngrok-free.app/v1/chat/completions"
MODEL_NAME = "llama-3.2-1b-instruct:2"

# HTML UI
chat_ui_html = """
<div id="lmstudio-chat" style="font-family: Arial; max-width: 600px;">
  <h3>LM Studio Chat Interface (ngrok)</h3>
  <textarea id="user-input" rows="3" style="width:100%;"></textarea><br><br>
  <button id="send-btn">Send</button>
  <div id="chat-log" style="margin-top: 20px; white-space: pre-wrap;"></div>
</div>

<script>
const sendBtn = document.getElementById("send-btn");
const userInput = document.getElementById("user-input");
const chatLog = document.getElementById("chat-log");

async function sendMessage() {
    const text = userInput.value.trim();
    if (!text) return;

    chatLog.innerHTML += "\\nYou: " + text;
    userInput.value = "";

    google.colab.kernel.invokeFunction('notebook.chat_to_lmstudio', [text], {});
}

sendBtn.addEventListener("click", sendMessage);
userInput.addEventListener("keydown", function(e) {
    if (e.key === "Enter" && !e.shiftKey) {
        e.preventDefault();
        sendMessage();
    }
});
</script>
"""
HTML(chat_ui_html)


In [ ]:
def chat_to_lmstudio(user_text):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "user", "content": user_text}
        ]
    }
    try:
        response = requests.post(LMSTUDIO_API_URL, json=payload, timeout=30)
        data = response.json()
        reply = data["choices"][0]["message"]["content"]
    except Exception as e:
        reply = f"[Error] {e}"

    output.eval_js(f'chatLog.innerHTML += "\\nLM Studio: {reply}";')

# JS → Python 함수 연결
output.register_callback('notebook.chat_to_lmstudio', chat_to_lmstudio)


In [ ]:
from IPython.display import HTML

chat_ui_html = """
<style>
/* 채팅 전체 컨테이너 */
#lmstudio-chat {
  font-family: 'Arial', sans-serif;
  max-width: 700px;
  margin: auto;
  background: #f4f7f9;
  border-radius: 15px;
  padding: 20px;
  box-shadow: 0 8px 20px rgba(0,0,0,0.1);
}

/* 타이틀 */
#chat-title {
  text-align: center;
  font-size: 1.4rem;
  margin-bottom: 10px;
  font-weight: bold;
  color: #444;
}

/* 채팅 Canvas 영역 */
#chat-canvas {
  width: 100%;
  height: 400px;
  background: white;
  border-radius: 10px;
  border: 1px solid #ddd;
  overflow-y: auto;
  padding: 10px;
  box-sizing: border-box;
}

/* 말풍선 스타일 */
.chat-bubble {
  display: inline-block;
  padding: 10px 14px;
  margin: 6px 0;
  border-radius: 10px;
  max-width: 80%;
  word-wrap: break-word;
}

.user-bubble {
  background: #4cafef;
  color: white;
  align-self: flex-end;
}

.ai-bubble {
  background: #ececec;
  color: #333;
}

/* 입력창 스타일 */
#input-container {
  display: flex;
  margin-top: 10px;
}

#user-input {
  flex: 1;
  padding: 10px;
  font-size: 1rem;
  border-radius: 8px;
  border: 1px solid #ccc;
}

#send-btn {
  background: #4cafef;
  color: white;
  border: none;
  padding: 10px 16px;
  margin-left: 8px;
  border-radius: 8px;
  cursor: pointer;
}
#send-btn:hover {
  background: #3a9bdc;
}
</style>

<div id="lmstudio-chat">
  <div id="chat-title">💬 LM Studio Chat</div>
  <div id="chat-canvas"></div>
  <div id="input-container">
    <textarea id="user-input" rows="2" placeholder="메시지를 입력하세요..."></textarea>
    <button id="send-btn">Send</button>
  </div>
</div>

<script>
const chatCanvas = document.getElementById("chat-canvas");
const sendBtn = document.getElementById("send-btn");
const userInput = document.getElementById("user-input");

function addMessage(text, sender) {
    const bubble = document.createElement("div");
    bubble.classList.add("chat-bubble");
    bubble.classList.add(sender === "user" ? "user-bubble" : "ai-bubble");
    bubble.innerText = text;
    chatCanvas.appendChild(bubble);
    chatCanvas.scrollTop = chatCanvas.scrollHeight;
}

async function sendMessage() {
    const text = userInput.value.trim();
    if (!text) return;
    addMessage(text, "user");
    userInput.value = "";
    google.colab.kernel.invokeFunction('notebook.chat_to_lmstudio', [text], {});
}

sendBtn.addEventListener("click", sendMessage);
userInput.addEventListener("keydown", function(e) {
    if (e.key === "Enter" && !e.shiftKey) {
        e.preventDefault();
        sendMessage();
    }
});

// Python에서 호출하는 함수 (AI 응답 표시)
window.displayAIResponse = function(reply) {
    addMessage(reply, "ai");
};
</script>
"""

HTML(chat_ui_html)


In [ ]:
from google.colab import output
import requests

def chat_to_lmstudio(user_text):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "user", "content": user_text}
        ]
    }
    try:
        response = requests.post(LMSTUDIO_API_URL, json=payload, timeout=30)
        data = response.json()

        if "choices" in data:
            reply = data["choices"][0]["message"]["content"]
        elif "error" in data:
            reply = f"[Error] {data['error']}"
        else:
            reply = "[Error] Unexpected API response"

    except Exception as e:
        reply = f"[Error] {e}"

    # JS 함수 호출해서 AI 응답을 Canvas에 표시
    output.eval_js(f'displayAIResponse({repr(reply)})')

output.register_callback('notebook.chat_to_lmstudio', chat_to_lmstudio)
